# projeto 1 : tratando e inserindo dados de gorjetas no postgresql

In [ ]:
# bibliotecas
import pandas as pd
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv

In [ ]:
# carrega as variaveis do arquivo .env
load_dotenv()  

# trazendo as variaveis
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")


# engine com sqlalchemy
engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

# testando a conexao(engine) com o banco de dados
try:
    with engine.connect() as conn:
        conn.execute(text("SELECT 1"))
    print("conexao com postgresql com sucesso")
except Exception as e:
    print("erro na conexao", e)
    raise

In [ ]:
# carrega o arquivo csv
caminho_do_arquivo = "tips.csv"
df = pd.read_csv(caminho_do_arquivo, encoding="utf-8-sig")
df.columns

In [ ]:
# mantem as colunas de interesse
df = df[['total_bill', 'tip', 'sex', 'day', 'time', 'size']] 

In [ ]:
# renomear as colunas
df.rename(columns={
    'total_bill': 'valor_total',
    'tip': 'gorjeta',
    'sex': 'sexo',
    'day': 'dia_semana_en',
    'time': 'horario_en',
    'size' : 'qtde_pessoas'
}, inplace=True)
df.head()

In [ ]:
# removendo linhas que não tem valor nas colunas mais importantes
df.dropna(subset=['valor_total', 'gorjeta'], inplace=True)

In [ ]:
# convertendo os formatos de algumas colunas
df = df.astype({
    'valor_total': 'float',
    'gorjeta': 'float',
    'qtde_pessoas': 'int'
})
df.head()

In [ ]:
# criando a tabela/zerando os dados antigos nela
create_table_sql = """
CREATE TABLE IF NOT EXISTS gorjetas (
    valor_total     FLOAT,
    gorjeta         FLOAT,
    sexo            VARCHAR(10),
    dia_semana_en   VARCHAR(5),
    horario_en      VARCHAR(10),
    qtde_pessoas    INTEGER
)
"""

with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.execute(text("TRUNCATE gorjetas"))     # zera a tabela
    conn.commit()

print("tabela gorjetas está ok")

In [ ]:
# enviando o dataframe para a tabela, com append e não replace
df.to_sql(
    'gorjetas',
    engine,
    if_exists='append',  # não sobrescreve, só adiciona
    index=False
)

print(f"{len(df)} registros inseridos com sucesso")